# LIBRERIAS

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=fbee081146abff4be829921a5c49da15929f52249e1a8cea61e62753e0fe7db0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
# from pyspark library import sql
from pyspark import sql

In [3]:
from pyspark.sql.functions import udf

In [4]:
import string

In [5]:
import pandas as pd

In [6]:
from textblob import TextBlob
from IPython.display import Markdown, display

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
import nltk

## DATOS

In [10]:
df_completo = pd.read_csv('/content/20220508_hepatitis (1).csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               50002 non-null  int64  
 1   conversation_id  50002 non-null  int64  
 2   created_at       50002 non-null  object 
 3   date             50002 non-null  object 
 4   time             50002 non-null  object 
 5   timezone         50002 non-null  int64  
 6   user_id          50002 non-null  int64  
 7   username         50002 non-null  object 
 8   name             50001 non-null  object 
 9   place            34 non-null     object 
 10  tweet            50002 non-null  object 
 11  language         50002 non-null  object 
 12  mentions         50002 non-null  object 
 13  urls             50002 non-null  object 
 14  photos           50002 non-null  object 
 15  replies_count    50002 non-null  int64  
 16  retweets_count   50002 non-null  int64  
 17  likes_count 

In [14]:
(df_completo.isnull().sum()/50002)*100

id                   0.000000
conversation_id      0.000000
created_at           0.000000
date                 0.000000
time                 0.000000
timezone             0.000000
user_id              0.000000
username             0.000000
name                 0.002000
place               99.932003
tweet                0.000000
language             0.000000
mentions             0.000000
urls                 0.000000
photos               0.000000
replies_count        0.000000
retweets_count       0.000000
likes_count          0.000000
hashtags             0.000000
cashtags             0.000000
link                 0.000000
retweet              0.000000
quote_url           92.770289
video                0.000000
thumbnail           84.996600
near               100.000000
geo                100.000000
source             100.000000
user_rt_id         100.000000
user_rt            100.000000
retweet_id         100.000000
reply_to             0.000000
retweet_date       100.000000
translate 

In [ ]:
df_completo.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [ ]:
# Eliminar columnas

df_completo.drop(columns=['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone','user_id', 'username', 'name', 'place', 'language', 'mentions','urls', 'photos', 'replies_count', 'retweets_count', 'likes_count','hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video','thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt','retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src','trans_dest'],inplace=True)

In [ ]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   50002 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB


In [ ]:
# Eliminar filas

df_mini = df_completo.drop(range(0, 49000),axis=0)

In [ ]:
df_mini.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 49000 to 50001
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   1002 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


In [ ]:
# Creating a SparkSession
spark_session = sql.SparkSession.builder.appName("pdf to sdf").getOrCreate()

In [ ]:
# Converting the pandas dataframe in to spark dataframe
spark_DataFrame = spark_session.createDataFrame(df_mini)

In [ ]:
#printing the dataframe
spark_DataFrame.show()

+--------------------+
|               tweet|
+--------------------+
|Making my point a...|
|Here's another on...|
|Could the fact th...|
|🔸 Feria de Segur...|
|EEUU cree adenovi...|
|@NUnl SARS-CoV-2 ...|
|Acute hepatitis: ...|
|Ukraine Governor ...|
|#Internacional | ...|
|Singapore confirm...|
|And nearly all th...|
|@DeMariekeK @degy...|
|Children Hepatiti...|
|@DrAseemMalhotra ...|
|I don’t expect th...|
|@macroliter Human...|
|#Pacientes | @Zap...|
|Desmienten presen...|
|La literatura cie...|
|SARS-CoV-2 vaccin...|
+--------------------+
only showing top 20 rows



# ANALISIS DE SENTIMIENTOS SIN ETIQUETAS CON TextBlob

In [ ]:
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [ ]:
for tweet in df_mini['tweet']:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("Neutral", color="grey")
        print("")

Making my point again regarding the recent string of hepatitis case bc many folks have miscommunicated my words and thread - Adv nor COVID should be ruled out 100%. We should still do our best to collect data. Differential diagnosis should consider ALL options.
Sentiment(polarity=0.5, subjectivity=0.35000000000000003)


<span style='color:green'>Positive</span>

Here's another one making out its covid causing fulminant hepatitis
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>


Could the fact the U.K. is a World leader for Covid infections be why we’re also leading the World in child hepatitis cases🤔  https://t.co/GxyyQRNhdh
Sentiment(polarity=0.0, subjectivity=0.0)


...

<span style='color:grey'>Neutral</span>


España confirma ocho casos de hepatitis infantil aguda, cuatro por adenovirus  https://t.co/Yglmbi7J9S
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>


Hepatitis raras infantiles. Con mi compañera @cglucio tratamos de recopilar lo que se sabe sobre esta nueva infección: síntomas, teorías sobre su origen y casos en el mundo @elmundosalud  https://t.co/sqXlFQ2S8H
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>Neutral</span>

In [ ]:
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative

In [ ]:
sentiment = udf(apply_blob)

In [ ]:
spark_DataFrame.withColumn("sentiment", sentiment(spark_DataFrame['tweet'])).show()

+--------------------+---------+
|               tweet|sentiment|
+--------------------+---------+
|Making my point a...|      1.0|
|Here's another on...|      0.0|
|Could the fact th...|      0.0|
|🔸 Feria de Segur...|      0.0|
|EEUU cree adenovi...|      0.0|
|@NUnl SARS-CoV-2 ...|      0.0|
|Acute hepatitis: ...|      1.0|
|Ukraine Governor ...|      0.0|
|#Internacional | ...|      0.0|
|Singapore confirm...|      1.0|
|And nearly all th...|      1.0|
|@DeMariekeK @degy...|      0.0|
|Children Hepatiti...|      1.0|
|@DrAseemMalhotra ...|      0.0|
|I don’t expect th...|      1.0|
|@macroliter Human...|      0.0|
|#Pacientes | @Zap...|      0.0|
|Desmienten presen...|      0.0|
|La literatura cie...|      1.0|
|SARS-CoV-2 vaccin...|      0.0|
+--------------------+---------+
only showing top 20 rows



In [ ]:
tr = spark_DataFrame.withColumn("sentiment", sentiment(spark_DataFrame['tweet']))

In [ ]:
tr.toPandas().to_csv('training_tweets.csv')

# ANALISIS DE SENTIMIENTOS CON ETIQUETAS CON NLTK

In [ ]:
comentarios_df = pd.read_csv('/content/training_tweets.csv')

In [ ]:
comentarios_df.shape

(1002, 3)

In [ ]:
comentarios_df.head(5)

,Unnamed: 0,tweet,sentiment
0,0,Making my point again regarding the recent str...,1.0
1,1,Here's another one making out its covid causin...,0.0
2,2,Could the fact the U.K. is a World leader for ...,0.0
3,3,🔸 Feria de Seguridad Vial y Salud este miércol...,0.0
4,4,EEUU cree adenovirus pudo causar casos de hepa...,0.0


In [ ]:
comentarios_df.columns

Index(['Unnamed: 0', 'tweet', 'sentiment'], dtype='object')

In [ ]:
#comentarios_df = comentarios_df.drop(['COMMENT_ID', 'AUTHOR', 'DATE'], axis=1)
#comentarios_df = comentarios_df.drop(['AUTHOR'], axis=1)
#comentarios_df = comentarios_df.drop(['DATE'], axis=1)

comentarios_df = comentarios_df.drop(comentarios_df.columns[[0]], axis=1)

comentarios_df.head(5)

,tweet,sentiment
0,Making my point again regarding the recent str...,1.0
1,Here's another one making out its covid causin...,0.0
2,Could the fact the U.K. is a World leader for ...,0.0
3,🔸 Feria de Seguridad Vial y Salud este miércol...,0.0
4,EEUU cree adenovirus pudo causar casos de hepa...,0.0


In [ ]:
#comentarios_df.drop([0],axis=0)

In [ ]:
comentarios_df = comentarios_df[['sentiment','tweet']]

comentarios_df.head(5)

,sentiment,tweet
0,1.0,Making my point again regarding the recent str...
1,0.0,Here's another one making out its covid causin...
2,0.0,Could the fact the U.K. is a World leader for ...
3,0.0,🔸 Feria de Seguridad Vial y Salud este miércol...
4,0.0,EEUU cree adenovirus pudo causar casos de hepa...


In [ ]:
comentarios_df.isnull().values.any()

False

In [ ]:
#para sacar las palabras no importantes

nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#vamos a crear una función que haga todo lo que hicimos hasta acá

def procesar_texto(cadena):
    sinpunt = [c for c in cadena if c not in string.punctuation]
    sinpunt = ''.join(sinpunt)
    palabras = sinpunt.split()
    cadena_cleaned = [palabra for palabra in palabras if palabra.lower() not in stopwords.words('spanish')]
    return cadena_cleaned

In [ ]:
#aplicaremos la función sobre df

comentarios_df['tweet'].head(5).apply(procesar_texto)

0    [Making, my, point, again, regarding, the, rec...
1    [Heres, another, one, making, out, its, covid,...
2    [Could, the, fact, the, UK, is, World, leader,...
3    [🔸, Feria, Seguridad, Vial, Salud, miércoles, ...
4    [EEUU, cree, adenovirus, pudo, causar, casos, ...
Name: tweet, dtype: object

In [ ]:
transformador = CountVectorizer(analyzer=procesar_texto).fit(comentarios_df['tweet'])

In [ ]:
# PRIMERO : dividir mensajes de entrenamiento y mensajes de prueba, con etiquetas

# genero cuatro variables correspondietnes 

com_entr, com_pru, etq_entr, etq_pru = train_test_split(comentarios_df['tweet'], comentarios_df['sentiment'], test_size=0.3)

In [ ]:
pipeline = Pipeline([
    ('vectorizar', CountVectorizer(analyzer=procesar_texto)), #primera operación
    ('transformar', TfidfTransformer()),
    ('clasificar', RandomForestClassifier())
])

In [ ]:
# SEGUNDO : entrenar el pipeline con los mensajes de entrenamiento

pipeline.fit(com_entr, etq_entr)

Pipeline(steps=[('vectorizar',
                 CountVectorizer(analyzer=<function procesar_texto at 0x7f714adb4dd0>)),
                ('transformar', TfidfTransformer()),
                ('clasificar', RandomForestClassifier())])

In [ ]:
# TERCERO : una vez entrenado el sistema, lo pruebo en sus predicciones

predicciones = pipeline.predict(com_pru)

In [ ]:
# CUARTO : informe de prueba

print(classification_report(etq_pru, predicciones))

              precision    recall  f1-score   support

         0.0       0.78      0.91      0.84       150
         1.0       0.65      0.75      0.70        99
         2.0       0.77      0.19      0.31        52

    accuracy                           0.73       301
   macro avg       0.74      0.62      0.62       301
weighted avg       0.74      0.73      0.70       301

